In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64
#### FIX ME #####
# imported AnimalShelter from the animal_shelter crud methods
from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# Added the username and password from the CRUD file I created 
username = "aacuser"
password = "mongoPW"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # added Grazioso's logo
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date

# Create the position of the logo to be centered
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([   
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                            height = 250, width = 251))], href = 'https://www.snhu.edu/', target = '_blank'), #creating size of the image and the link to snhu when the logo is clicked on
    html.Center(html.B(html.H1('Trevor Skillman SNHU CS-340 Dashboard'))), # added the unique ID with my name and class number
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        # Decided to keep it simple and add radio buttons as the filtering option
        dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'}, #water rescue label and value
            {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain Rescue'}, #Mountain Rescue label and value
            {'label': 'Disaster Rescue/Individual Tracking', 'value': 'Disaster Rescue'}, # Disaster rescue lable and value
            {'label': 'Reset', 'value': 'Reset'}, # reset label and value
        ],
            value='ALL',
            labelStyle={'display': 'inline-block'}
    ),
        ),
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
# Copy and pasted my data table from the previous week and made less interactive features so the user would not feel overwhelmed
        editable=False,
        style_cell={'textAlign': 'left'},
        style_cell_conditional=[
            {
                'if': {'column_id': j},
                'textAlign': 'left'
            }for j in ['age_upon_outcome', 'animal_id', 'animal_type', 'breed',
                       'color', 'date_of_birth', 'datetime', 'monthyear']
        ],
        style_data_conditional=[
            {
                'if': {'row_index': 'even'},
                'backgroundColor': 'rgb(50,50,50)',
                'color': 'white'
            }
        ],
        style_header={
            'backgroundColor': 'rgb(30,30,30)',
            'fontWeight': 'bold',
            'color': 'white'
        },
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### Adding code for the interactive radio buttons to be used
        if(filter_type == 'Water Rescue'): # If filter_type is true to Water Rescue
        #it will read the shelter parameter to output the querie to what type of dog is available
           df = pd.DataFrame(shelter.read_all({
               "animal_type": "Dog", # read the animal type as a dog
               "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, # read the available breeds for a water rescue
               "sex_upon_outcome": "Intact Female", # Grazio asked that the dog hads to be an intact female, this will assure we only pick out intact female for the rescue
               "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0} # using $gte means the field is >= specified value,
               # using $lte means the field is <= specified value
           }))
        elif(filter_type == 'Mountain Rescue'): # else if filter_type is true to the Mountain rescue
            # we will instatiate reading the db
            df = pd.DataFrame(shelter.read_all({
                "animal_type": "Dog", # Grazio asked for the rescue animal to be a dog
                "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, # Dogs available for mountain rescue
                "sex_upon_outcome": "Intact Male", # they must be an intact male
                "age_upon_outcome_in_weeks": {"$gte":26, "$lte":156} # age upon outcome in weeks and making sure we dont go under or over on the value needed
            }))
        elif(filter_type == 'Disaster Rescue'): # else if filter_type is true to Disaster rescue 
            # we read the db for the query in question
            df = pd.DataFrame(shelter.read_all({
                "animal_type": "Dog", # query to make sure it is a animal type of a Dog
                "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, # Dogs avaliable for disater rescue
                "sex_upon_outcome": "Intact Male", # query to make sure it is an intact male
                "age_upon_outcome_in_weeks": {"$gte":20, "$lte":300} # query to make sure it is within a specific age range
            }))
        else:
            # else will reset the radio button filters to see all animals within the database
            df = pd.DataFrame.from_records(shelter.read_all({}))
                            
        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewPie):
    dffPie = pd.DataFrame.from_dict(viewPie)
    # Created a piechart code
    return [
        dcc.Graph(  
            id = 'graph_id',
            figure = px.pie(dffPie, # reads the dictionary 
                            title = 'Dog Breeds Available',# Creating the title of the pie chart
                            values = 'age_upon_outcome_in_weeks', # the value of age upon outcome in weeks 
                            names = 'breed' # the breed type that is available for the perticular rescue
                )
            )    
        ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
# Reading the dataFrame dictionary 
    dff = pd.DataFrame.from_dict(viewData)
    return[
        # returning the map with the lat/long location for the animal shelter in austin texas
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # marker creation 
            dl.Marker(position=[dff.loc[0,'location_lat'], dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    # information that shows what is inside of the marker when the marker is clicked, that being the animal name 
                    html.H1("Animal name"),
                    html.P(dff.loc[0,'name'])
                ])
            ])
        ])
    ]


app